In [ ]:
from datetime import datetime
import pandas_gbq
import plotly.express as px

In [ ]:
df = pandas_gbq.read_gbq("SELECT * FROM `nais-analyse-prod-2dcc.deploys.deploys_prod_aggregated`", project_id='nais-analyse-prod-2dcc')

In [ ]:
#Filter out canaries and auto-deployed system apps
df = df[df['application'] != 'nais-deploy-canary']
df = df[df['application'] != 'testapp-storage']
df = df[df['application'] != 'kafkarator-canary']

In [ ]:
df['weeknr'] = df['deployed_timestamp'].dt.isocalendar().week.astype(str)
df['weeknr'] = df['weeknr'].apply(lambda x: x.zfill(2))
df['year'] = df['deployed_timestamp'].dt.isocalendar().year.astype(str)
df['monthnr'] = df['deployed_timestamp'].dt.month.astype(str)
df['monthnr'] = df['monthnr'].apply(lambda x: x.zfill(2))
df['week'] = df['year'] + '-' + df['weeknr']
df['month'] = df['year'] + '-' + df['monthnr']

In [ ]:
start = df['deployed_timestamp'].min()

In [ ]:
df_month_prod = df.groupby('month').size().reset_index(name='antall')
fig_month_prod = px.bar(df_month_prod, x='month', y='antall')
fig_month_prod.update_xaxes(type='category')
fig_month_prod.show()

In [ ]:
df_year = df.groupby('year').size().reset_index(name='antall')
df_year = df_year[df_year['year'].astype(int)>2014]

In [ ]:
df_weekly_avg = df.groupby(['weeknr', 'year']).size().reset_index(name='antall') \
       .groupby('year').agg(snitt_deploys_per_uke=('antall', 'mean')).reset_index()
df_weekly_avg['snitt_deploys_per_uke'] = df_weekly_avg['snitt_deploys_per_uke'].round(0)
df_weekly_avg = df_weekly_avg[df_weekly_avg['year'].astype(int)>2014]

In [ ]:
description = f'''Deploy-dataproduct (prod) inneholder deploys til prod lest ut fra `vera` og `devrapid` i perioden fra {start.strftime("%Y-%m-%d kl %H:%M:%S")} til nå. Dataproduktet oppdateres i sanntid.'''

In [ ]:
#| output: true
from IPython.display import Markdown
Markdown(description)

# Eksempler
## Deploys til produksjon per måned

I denne grafen har vi telt opp antall deploys til produksjon per år.

In [ ]:
#| output: true

import numpy as np
import matplotlib.pyplot as plt

fig_year = plt.figure(figsize=(14, 6), dpi=80)

with plt.xkcd():
    plt.bar(df_year['year'], df_year['antall'], color='grey')
    plt.title('Deploys to production in NAV')
    plt.xlabel('year')
    plt.ylabel('deploys')

I denne grafen har vi gjennomsnittlig antall deploys til produksjon per uke.

In [ ]:
#| output: true

fig_yearly_avg = plt.figure(figsize=(14, 6), dpi=80)

with plt.xkcd():
    plt.bar(df_weekly_avg['year'], df_weekly_avg['snitt_deploys_per_uke'], color='grey')
    plt.title('Average weekly deploys to production in NAV')
    plt.xlabel('year')
    plt.ylabel('deploys per week')